# PHYS 210 Mini-Project 05
### Diffusion
Due Wed, Dec 07, 2022 - 9am

In [ ]:
# Main code and animation here
# YOUR CODE HERE
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from scipy.optimize import curve_fit

# Constants:
L = 1  # Size of 'box'
D = 1e-3
# We will include the phantom grid perimeter in our number of points
# to minimize additional complicated bookkeeping
N = 101

dt = 1e-2
dx = L / N

# tmax = 11
tmax = 100
steps = int(tmax / dt) + 1

# Choose some times to make plots, given in step numbers:
plotsteps = np.arange(0, tmax + 0.2, 0.2)
plotsteps /= dt
plotsteps = plotsteps.astype(int)

# Choose some times for 1D C(x, y, t), given in step numbers:
plotsteps2 = np.linspace(0.01, 10, num=30)
plotsteps2 /= dt
plotsteps2 = plotsteps2.astype(int)

# Create initial conditions:
C = np.zeros((N, N))

# Set particles in a blob in the center:
C[N // 2][N // 2] = 10

# animation setup
fig = plt.figure()
fig.set_size_inches(10, 8)
fig.set_dpi(256)
plt.title("Evolution of C(x, y, t) from 0 ~ 100 second")
ims = []  # list to store pcolormeshes

# Our initial number of particles is the sum of C across
# all of our non-phantom grid points
isum = np.sum(C[1:-1][1:-1])
k = D * dt / dx**2
# print(k)

# C-prime (the concentrations after each time step)
Cp = np.zeros((N, N))

# List to store data to fit in the gaussian curve
time_C_1D = []
stddev_arr = []
x_data = np.arange(N)
concentration_sum = []


def Gaussian_fit(x, amp, mu, std):
    """
    Function of a Gaussian Curve used to fit the 1D C(x, y, t)
    amp = constant related to the amplitude of the curve
    mu = mean
    std = standard deviation
    """
    return amp / std * np.sqrt(2 * np.pi) * np.exp(-0.5 * (x - mu)**2 / std**2)


# time evolution using for loop (p)
for p in range(steps):
    t = round(p * dt, 2)  # time at this particular step
    # 1 Implement the diffision equation at all non-phantom grid points:
    Cp[1:-1, 1:-1] = C[1:-1, 1:-1] + k * (
        ((C[2:, 1:-1] - 2 * C[1:-1, 1:-1] + C[:-2, 1:-1]) +
         (C[1:-1, 2:] - 2 * C[1:-1, 1:-1] + C[1:-1, :-2])))

    # The phantom grid points are just reflections of their neighbours
    # on the other side of the boundary (to maintain dC/dx = 0):
    Cp[0, :] = Cp[1, :]
    Cp[:, 0] = Cp[:, 1]
    Cp[-1, :] = Cp[-2, :]
    Cp[:, -1] = Cp[:, -2]

    # Update C to be Cp and then Cp will get overwritten on the next loop:
    C, Cp = Cp, C

    # 2 append to animated array
    if p in plotsteps:
        img = plt.pcolormesh(C)
        ims.append((img, ))

    # 3 Fit the gaussian curve to 30 1D frames between 0.01 and 10s
    if p in plotsteps2:
        y = C[:][N // 2]
        # plt.plot(y, label="t = %g" % (t))
        # full width of the gaussian curve
        full_width = np.where(y != 0)
        # standard deviation of the gaussian curve as 34.1% of its width
        s = (np.max(full_width) - np.min(full_width)) * 0.341
        popt, pcov = curve_fit(Gaussian_fit,
                               xdata=x_data,
                               ydata=y,
                               p0=[max(y) * np.sqrt(2 * np.pi) * s, 50, s])
        # divide the fitted standard deviation by 100
        # to fit back to the 1x1 grid
        # and append it to an array to plot the data later
        stddev_arr.append(popt[2] / 100)
        # plt.plot(x_data,
        #          Gaussian_fit(x_data, *popt), '--k',
        #          label="fit - t = %g" % (t))
        time_C_1D.append(t)  # timestamp for each 1D array
        # print(t)
        # print(C[:][N // 2])

    # 4 data to plot the total concentration vs t
    concentration_sum.append(np.sum(C[1:-2][1:-2]))

# # print(C_1Dim)
# plt.legend()
# plt.show()
# plt.close()
# print("start animation.")
# create animation
imani = animation.ArtistAnimation(fig, ims, repeat_delay=10)
plt.colorbar(img)
imani.save('Cxyt_Evolution.gif', writer=animation.PillowWriter(fps=60))
plt.close()  # Prevents a stray plot from appearing
del ims  # Release crucial Cocal memory related to these objects
del imani  #
HTML('<img src="Cxyt_Evolution.gif">')

# If the boundary conditions are done correctly, the number of particles
# (the integral of C), should be constant. Check:
esum = np.sum(C[1:-1][1:-1])
print("initial and final integrals of concentration: {:.2f} and {:.2f}".format(
    isum, esum))

In [ ]:
# Plot of Gaussian width vs sqrt(t) here:
# YOUR CODE HERE

plt.scatter(np.sqrt(time_C_1D), stddev_arr, label="stddev")
newt = np.arange(0, 10, 0.01)
plt.plot(np.sqrt(newt), np.sqrt(2 * D * newt), label="sqrt(2Dt)")
plt.xlabel("Square root of time (s)")
plt.ylabel("Gaussian Width")
plt.title("Gaussian wiidth vs sqrt(t)")
plt.legend()
plt.show()
plt.close()

In [ ]:
# Plot of particle count (sum of non-phantom C) vs t here:
# YOUR CODE HERE

plt.plot(np.arange(0, tmax+dt, dt), concentration_sum)
plt.ylim([0, 11])
plt.xlim([0, tmax])
plt.xlabel("Time (s)")
plt.ylabel("Total Concentration")
plt.title("Total concentration of C(x, y) vs t")
plt.show()
plt.close()

### Your answers to text questions for Part 5 questions here. There is an ungraded, un-timed and not-style-checked cell below for you to do your work for anwering these questions

Answer the following questions in the space provided in this cell.

**5a)** What happens to the animation when dt is too large?



**5b)** What is the value of dt where the simulation diverges? 



**5c)** What is the value of $dt \, D/dx^2$ where the simulation starts to diverge? 



**5d)** What happens to the sums you generate for question (4) when the simulation diverges?  



**5e)** If you use a grid of only 51x51 points, where is the threshold for dt to converge now? 




YOUR ANSWER HERE

**5a)** The solution will diverge when dt is too large

**5b)** dt = 2.5e-2 = 0.025

**5c)** dt D / dx^2 = 2.5e-2 * 1e-3 / (L/N)^2, L = 1, N = 101, dt D / dx^2 = 0.255025

**5d)** The sum is supposed to remain 10 when the system converges. In this case, the sum of C(x, y, t) increases to inf from 10 as dt increases from 2.5e-2.

**5e)** dt should be <= 7.5e-2 for a system with 51x51 points to converge

In [ ]:
# Main code and animation here
# YOUR CODE HERE
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from scipy.optimize import curve_fit


def funct(N, dt):
    print("dt = ", dt)
    # Constants:
    L = 1  # Size of 'box'
    D = 1e-3
    # We will include the phantom grid perimeter in our number of points
    # to minimize additional complicated bookkeeping

    dx = L / N

    tmax = 11
    # tmax = 100
    steps = int(tmax / dt) + 1

    # Choose some times for 1D C(x, y, t), given in step numbers:
    plotsteps2 = np.linspace(0.01, 10, num=30)
    plotsteps2 /= dt
    plotsteps2 = plotsteps2.astype(int)

    # Create initial conditions:
    C = np.zeros((N, N))

    # Set particles in a blob in the center:
    C[N // 2][N // 2] = 10

    # Our initial number of particles is the sum of C across
    # all of our non-phantom grid points
    isum = np.sum(C[1:-1][1:-1])
    k = D * dt / dx**2
    # print(k)

    # C-prime (the concentrations after each time step)
    Cp = np.zeros((N, N))

    # time evolution using for loop (p)
    for p in range(steps):
        t = round(p * dt, 2)  # time at this particular step
        # 1 Implement the diffision equation at all non-phantom grid points:
        Cp[1:-1, 1:-1] = C[1:-1, 1:-1] + k * (
            ((C[2:, 1:-1] - 2 * C[1:-1, 1:-1] + C[:-2, 1:-1]) +
             (C[1:-1, 2:] - 2 * C[1:-1, 1:-1] + C[1:-1, :-2])))

        # The phantom grid points are just reflections of their neighbours
        # on the other side of the boundary (to maintain dC/dx = 0):
        Cp[0, :] = Cp[1, :]
        Cp[:, 0] = Cp[:, 1]
        Cp[-1, :] = Cp[-2, :]
        Cp[:, -1] = Cp[:, -2]

        # Update C to be Cp and then Cp will get overwritten on the next loop:
        C, Cp = Cp, C

        # 3 Fit the gaussian curve to 30 1D frames between 0.01 and 10s
        if p in plotsteps2:
            y = C[:][N // 2]
            plt.plot(y, label="t = %g" % (t))
    plt.show()
    plt.close()
    # If the boundary conditions are done correctly, the number of particles
    # (the integral of C), should be constant. Check:
    esum = np.sum(C[1:-1][1:-1])
    print("initial and final integrals of concentration: {:.2f} and {:.2f}".
          format(isum, esum))
    return


# funct(51, 7.5e-2)
# funct(51, 9.6e-2)
# funct(51, 9.8e-2)

# Acknowledgements

In the cell below, please describe the role of **anyone other than yourself** who contributed to the work shown in this notebook.

Its ok to get help from us and classmates! Please get in the habit of acknowledging such contributions.

If you want to refer to a classmate, please use only their cocalc email-id and not their name - or you could just say something like: "a classmate gave me the idea to use xxx feature to solve yyy problem."


_Acknowledgements here:_



# Extension Code and Description
All solution code for the main project question should appear in the two main cells above above. Project extensions go in the cell "cell-extension" immediately below and the descriptions of your extension go in the cell below that.

In [ ]:
# OPTIONAL project extension here
# These can call functions in your code above if desired



_In this cell, please describe any new language features or project extension you have implemented:_




# Grading cells
The cells below marked as "grade use only" are created as placeholders so that we can provide a manual grade and comments for each category. 

Exceptions are the "2. Style" test, which has an associated autograder test that you can run to check style and the timing cell "cell-optimization0", which you can use to test your code execution time.

In [ ]:
# 1. Code execution (grader use only)

In [ ]:
# 2. Style: pep8 (see note below regarding use of the Format button to fix many errors)
#
# Tests for pep8 returns warnings or errors. You may need to hit 'Save' after making changes for them to take effect.
nb_name = "project05.ipynb"
cells_to_check = []
stop_at = ['cell-extension']
# check_style2.py uses cells_to_check and nb_name
%run -i check_style2.py

Also note that you can use the Format button while in a code cell to automagically fix most pep8 errors (other than way too long print statements)

![](project02-format.png)

In [ ]:
# 3. Results (grader use only)

In [ ]:
# 4. Readability (grader use only)

In [ ]:
# 5. Plot (grader use only)

In [ ]:
# Check execution time
nb_name = "project05.ipynb"
cells_to_time = []
stop_at = ['cell-extension']
%run -i time_cells2.py

In [ ]:
# 5. Code optimization/timing (grader use only)

In [ ]:
# B2. New Functionality/Language features (grader use only)